In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
import time
from bs4 import BeautifulSoup

In [10]:
#url = "https://www.sofascore.com/tournament/football/england/premier-league/17#id:61627,tab:matches"
urls = {}
#urls['24/25'] = "https://www.sofascore.com/tournament/football/england/premier-league/17#id:61627,tab:matches"
urls['23/24'] = "https://www.sofascore.com/tournament/football/england/premier-league/17#id:52186,tab:matches"
#urls['22/23'] = "https://www.sofascore.com/tournament/football/england/premier-league/17#id:41886,tab:matches"
#urls['21/22'] = "https://www.sofascore.com/tournament/football/england/premier-league/17#id:41886,tab:matches"
#urls['20/21'] = "https://www.sofascore.com/tournament/football/england/premier-league/17#id:29415,tab:matches"

In [11]:
options = webdriver.ChromeOptions()
options.set_capability('goog:loggingPrefs', {"performance": "ALL", "browser": "ALL"})
# Adding user agent to look more like a real browser
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)
wait = WebDriverWait (driver,10)
driver.set_page_load_timeout(50)
driver.set_window_size(800, 600)

In [12]:
season_matches = {}
for season in urls:
    driver.get(urls[season])
    time.sleep(3)  # Allow page to load
    
    try:
        by_date_button = driver.find_element(By.XPATH, "//div[@data-tabid='date']")
        time.sleep(1)
        driver.execute_script("arguments[0].click();", by_date_button)
        print("Clicked 'By date' button")
        time.sleep(5)
    except Exception as e:
        print("Failed to click 'By date' button:", e)
    
    # Click "Results" button
    try:
        results_button = driver.find_element(By.XPATH, "//button[@data-tabid='0']")
        driver.execute_script("arguments[0].click();", results_button)
        print("Clicked 'Results' button")
        time.sleep(3)  # Wait for matches to load
    except Exception as e:
        print("Failed to click 'Results' button:", e)
    
    
    # Scroll to load all past matches
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    print ("last height: " , last_height)
    prev = 0
    while True:
        next_height = 0
        while next_height < last_height - 200:
            next_height = prev + 25
            driver.execute_script(f"window.scrollTo({prev}, {next_height});")
            time.sleep (0.1)
            prev = next_height
        # Wait for new matches to load
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        print ("new_height: ", new_height)
        if new_height == last_height:
            break  # Stop when no more matches load
        last_height = new_height
    
    time.sleep (5) # time to load
    soup = BeautifulSoup(driver.page_source, "html.parser")
    season_matches[season] = soup
driver.quit()  # Close Selenium

Clicked 'By date' button
Clicked 'Results' button
last height:  2040
new_height:  3480
new_height:  4920
new_height:  6360
new_height:  7800
new_height:  9240
new_height:  10680
new_height:  12120
new_height:  13560
new_height:  15000
new_height:  16440
new_height:  17880
new_height:  18800
new_height:  18800


In [13]:
match_links = {}
for season in season_matches:
    match_list = []
    for link in season_matches[season].find_all("a", href=True):
        href = link["href"]
        
        # Sofascore match URLs contain '/football/' and have IDs
        if "/football/match" in href and href.count("/") > 2:  
            full_url = f"https://www.sofascore.com{href}"
            print (full_url)
            match_list.append(full_url)
    
    # Remove duplicates
    match_links[season] = list(set(match_list))

for season in match_links:
    print (len(match_links[season]))


https://www.sofascore.com/football/match/west-ham-united-manchester-city/rM#id:11352571
https://www.sofascore.com/football/match/manchester-united-brighton-and-hove-albion/FsK#id:11352552
https://www.sofascore.com/football/match/liverpool-wolverhampton/dsU#id:11352565
https://www.sofascore.com/football/match/everton-arsenal/RY#id:11352546
https://www.sofascore.com/football/match/bournemouth-chelsea/Nskb#id:11352558
https://www.sofascore.com/football/match/tottenham-hotspur-sheffield-united/psI#id:11352574
https://www.sofascore.com/football/match/brentford-newcastle-united/Osab#id:11352549
https://www.sofascore.com/football/match/aston-villa-crystal-palace/hP#id:11352562
https://www.sofascore.com/football/match/luton-town-fulham/Tsxb#id:11352568
https://www.sofascore.com/football/match/nottingham-forest-burnley/gso#id:11352555
https://www.sofascore.com/football/match/newcastle-united-manchester-united/DYBe#id:12240571
https://www.sofascore.com/football/match/chelsea-brighton-and-hove-al

In [14]:
for season in match_links:
    season_name = season.split("/")
    season_full = season_name[0] + "_" + season_name[1]
    f = open (f"match_links_{season_full}.txt","w")
    for match in match_links[season]:
        match_id = match.split(":")[2].strip()
        f.write(match_id + "\n")
    f.close()